# Introduction aux réseaux de neurones : TD #2  -- Pistes de solutions
Matériel de cours rédigé par Pascal Germain, 2018
************

### Changer la taille et le nombre de filtres convolutifs

Ici, nous réutilisons la classe `UneArchiPourMNIST` pour obtenir 64 filtres de taille 9 chacun.

In [ ]:
mon_archi = UneArchiPourMNIST(nb_filtres=64, taille_noyau=9)
R = ReseauClassifGenerique(mon_archi, eta=0.1, alpha=0.1, nb_epoques=20, taille_batch=32)

### Ajouter une ou plusieurs couches de filtres convolutifs dans la première partie du réseau

Modifions la classe `UneArchiPourMNIST` afin d'ajouter une 2e couche de filtres  convolutifs.

In [ ]:
class Archi2CouchesCovolutives:
    def __init__(self, couche1_nb_filtres=16, couche1_taille_noyau=9, 
                       couche2_nb_filtres=32, couche2_taille_noyau=3):
        # Créons deux couches de convolution 
        self.modele_conv = nn.Sequential(
            # Première couche de convolution
            nn.Conv2d(1, couche1_nb_filtres, kernel_size=couche1_taille_noyau),
            nn.ReLU(),
            nn.MaxPool2d(2),
            # NOUVEAUTÉ: Deuxième couche de convolution
            nn.Conv2d(couche1_nb_filtres, couche2_nb_filtres, kernel_size=couche2_taille_noyau),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        
        # La convolution est suivie d'une couche de sortie
        # NOUVEAUTÉ: Calcul du nombre de neurones sur la couche pleinement connectée
        #            considérant la 2e couche de convolution
        nb_pixels = (28 - couche1_taille_noyau + 1) // 2 
        nb_pixels = (nb_pixels - couche2_taille_noyau + 1) // 2
        self.nb_neurones_du_milieu = couche2_nb_filtres * (nb_pixels**2)
        
        self.modele_plein = nn.Sequential(
            nn.Linear(self.nb_neurones_du_milieu, 10),
            nn.LogSoftmax(dim=1)
        )
        
    def propagation(self, x, apprentissage=False):
        # Propageons la «batch». Notez que nous devons redimensionner nos données consciencieusement
        x0 = x.view(-1, 1, 28, 28)
        x1 = self.modele_conv(x0)
        x2 = x1.view(-1, self.nb_neurones_du_milieu)
        x3 = self.modele_plein(x2)
        return x3
    
    def parametres(self):
        # Cette fonction doit retourner un tuple contenant toutes les variables à optimiser
        return self.modele_conv.parameters(), self.modele_plein.parameters()

In [ ]:
mon_archi = Archi2CouchesCovolutives(couche1_nb_filtres=16, couche1_taille_noyau=9,
                                     couche2_nb_filtres=32, couche2_taille_noyau=3)
R = ReseauClassifGenerique(mon_archi, eta=0.1, alpha=0.1, nb_epoques=20, taille_batch=32)

### Ajouter davantage de couches pleinement connectées dans la seconde partie du réseau

Reprenons l'architecture `Archi2CouchesCovolutives` ci-haut et ajoutons-y une couche cachée pleinement connectée

In [ ]:
class Archi2CouchesCovolutives2CouchesPC:
    def __init__(self, couche1_nb_filtres=16, couche1_taille_noyau=9, 
                       couche2_nb_filtres=32, couche2_taille_noyau=3,
                       couche3_nb_neurones=100):
        # Créons deux couches de convolution 
        self.modele_conv = nn.Sequential(
            # Première couche de convolution
            nn.Conv2d(1, couche1_nb_filtres, kernel_size=couche1_taille_noyau),
            nn.ReLU(),
            nn.MaxPool2d(2),
            # Deuxième couche de convolution
            nn.Conv2d(couche1_nb_filtres, couche2_nb_filtres, kernel_size=couche2_taille_noyau),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        
        # La convolution est suivie d'une couche cachée pleinement connectée
        nb_pixels = (28 - couche1_taille_noyau + 1) // 2
        nb_pixels = (nb_pixels - couche2_taille_noyau + 1) // 2
        self.nb_neurones_du_milieu = couche2_nb_filtres * (nb_pixels**2)
        
        self.modele_plein = nn.Sequential(
            # NOUVEAUTÉ: Nouvelle couche cachée pleinement connectée avec activation ReLU
            nn.Linear(self.nb_neurones_du_milieu, couche3_nb_neurones),  
            nn.ReLU(),
            # Couche de sortie
            nn.Linear(couche3_nb_neurones, 10),
            nn.LogSoftmax(dim=1)
        )
        
    def propagation(self, x, apprentissage=False):         
        # Propageons la «batch». Notez que nous devons redimensionner nos données consciencieusement
        x0 = x.view(-1, 1, 28, 28)
        x1 = self.modele_conv(x0)
        x2 = x1.view(-1, self.nb_neurones_du_milieu)
        x3 = self.modele_plein(x2)
        return x3
    
    def parametres(self):
        # Cette fonction doit retourner un tuple contenant toutes les variables à optimiser
        return self.modele_conv.parameters(), self.modele_plein.parameters()

In [ ]:
mon_archi = Archi2CouchesCovolutives2CouchesPC(couche1_nb_filtres=16, couche1_taille_noyau=9, 
                                               couche2_nb_filtres=32, couche2_taille_noyau=3)
R = ReseauClassifGenerique(mon_archi, eta=0.1, alpha=0.1, nb_epoques=20, taille_batch=32)

### Ajouter du «Dropout»

Reprenons l'architecture précedente et ajoutons du *Dropout* sur chacune des couches cachées. Notons qu'il est désormais essentiel de distinguer la phase **apprentissage** de la phase **évaluation** (voir les premières lignes de la  méthode `propagation`).

In [ ]:
class ArchiDropout2CouchesCovolutives2CouchesPC:
    def __init__(self, couche1_nb_filtres=16, couche1_taille_noyau=9, couche1_prob_dropout=0.5, 
                       couche2_nb_filtres=32, couche2_taille_noyau=3, couche2_prob_dropout=0.5, 
                       couche3_nb_neurones=100, couche3_prob_dropout=0.5):
        # Créons deux couches de convolution 
        self.modele_conv = nn.Sequential(
            # Première couche de convolution
            nn.Conv2d(1, couche1_nb_filtres, kernel_size=couche1_taille_noyau),
            nn.Dropout2d(couche1_prob_dropout), # <-- NOUVEAUTÉ
            nn.ReLU(), 
            nn.MaxPool2d(2),
            # Deuxième couche de convolution
            nn.Conv2d(couche1_nb_filtres, couche2_nb_filtres, kernel_size=couche2_taille_noyau),
            nn.Dropout2d(couche2_prob_dropout),  # <-- NOUVEAUTÉ
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        
        # La convolution est suivie d'une couche cachée pleinement connectée
        nb_pixels = (28 - couche1_taille_noyau + 1) // 2
        nb_pixels = (nb_pixels - couche2_taille_noyau + 1) // 2
        self.nb_neurones_du_milieu = couche2_nb_filtres * (nb_pixels**2)
        
        self.modele_plein = nn.Sequential(
            # Couche cachée pleinement connectée
            nn.Linear(self.nb_neurones_du_milieu, couche3_nb_neurones),  
            nn.Dropout(couche3_prob_dropout),  # <-- NOUVEAUTÉ
            nn.ReLU(),
            # Couche de sortie
            nn.Linear(couche3_nb_neurones, 10),
            nn.LogSoftmax(dim=1)
        )
        
    def propagation(self, x, apprentissage=False):
        # Réseau avec «dropout»: il est essentiel de distinguer la phase d'apprentissage
        # et la phase de prédiction!
        if apprentissage:   
            self.modele_conv.train()
            self.modele_plein.train()
        else:
            self.modele_conv.eval()
            self.modele_plein.eval()
          
        # Propageons la «batch». Notez que nous devons redimensionner nos données consciencieusement
        x0 = x.view(-1, 1, 28, 28)
        x1 = self.modele_conv(x0)
        x2 = x1.view(-1, self.nb_neurones_du_milieu)
        x3 = self.modele_plein(x2)
        return x3
    
    def parametres(self):
        # Cette fonction doit retourner un tuple contenant toutes les variables à optimiser
        return self.modele_conv.parameters(), self.modele_plein.parameters()

Comparativement aux expérimentations précédentes, remarquez que nous avons diminué le taux d'apprentissage (paramètre `eta`) et augmenter le nombre d'époques (paramètre `nb_epoques`) après avoir constaté expérimentalement que l'ajout de *dropout* ralentie la convergence de la valeur objectif.

In [ ]:
mon_archi = ArchiDropout2CouchesCovolutives2CouchesPC(
                    couche1_nb_filtres=16, couche1_taille_noyau=9, couche1_prob_dropout=0.5, 
                    couche2_nb_filtres=32, couche2_taille_noyau=3, couche2_prob_dropout=0.5, 
                    couche3_nb_neurones=100, couche3_prob_dropout=0.5
                )
R = ReseauClassifGenerique(mon_archi, eta=0.05, alpha=0.1, nb_epoques=40, taille_batch=32)

### Faire de la «Batchnorm»

Reprenons l'architecture précédente, en remplacant le *dropout* par la *batchnorm*.

In [1]:
class ArchiBatchnorm2CouchesCovolutives2CouchesPC:
    def __init__(self, couche1_nb_filtres=16, couche1_taille_noyau=9, 
                       couche2_nb_filtres=32, couche2_taille_noyau=3,
                       couche3_nb_neurones=100):
        # Créons deux couches de convolution 
        self.modele_conv = nn.Sequential(
            # Première couche de convolution
            nn.Conv2d(1, couche1_nb_filtres, kernel_size=couche1_taille_noyau),
            nn.ReLU(),
            nn.BatchNorm2d(couche1_nb_filtres),  # <-- NOUVEAUTÉ 
            nn.MaxPool2d(2),
            # Deuxième couche de convolution
            nn.Conv2d(couche1_nb_filtres, couche2_nb_filtres, kernel_size=couche2_taille_noyau),
            nn.ReLU(),
            nn.BatchNorm2d(couche2_nb_filtres),  # <-- NOUVEAUTÉ
            nn.MaxPool2d(2),
        )
               
        # La convolution est suivie d'une couche cachée pleinement connectée
        nb_pixels = (28 - couche1_taille_noyau + 1) // 2
        nb_pixels = (nb_pixels - couche2_taille_noyau + 1) // 2
        self.nb_neurones_du_milieu = couche2_nb_filtres * (nb_pixels**2)
        
        self.modele_plein = nn.Sequential(
            # Couche cachée pleinement connectée
            nn.Linear(self.nb_neurones_du_milieu, couche3_nb_neurones),  
            nn.BatchNorm1d(couche3_nb_neurones),  # <-- NOUVEAUTÉ
            nn.ReLU(),
            # Couche de sortie
            nn.Linear(couche3_nb_neurones, 10),
            nn.LogSoftmax(dim=1)
        )
        
        
    def propagation(self, x, apprentissage=False):
        # Réseau avec «batchnorm»: il est essentiel de distinguer la phase d'apprentissage
        # et la phase de prédiction!
        if apprentissage: #  
            self.modele_conv.train()
            self.modele_plein.train()
        else:
            self.modele_conv.eval()
            self.modele_plein.eval()
          
        # Propageons la «batch». Notez que nous devons redimensionner nos données consciencieusement
        x0 = x.view(-1, 1, 28, 28)
        x1 = self.modele_conv(x0)
        x2 = x1.view(-1, self.nb_neurones_du_milieu)
        x3 = self.modele_plein(x2)
        return x3
    
    def parametres(self):
        # Cette fonction doit retourner un tuple contenant toutes les variables à optimiser
        return self.modele_conv.parameters(), self.modele_plein.parameters()

In [ ]:
mon_archi = ArchiBatchnorm2CouchesCovolutives2CouchesPC(
                    couche1_nb_filtres=16, couche1_taille_noyau=9,
                    couche2_nb_filtres=32, couche2_taille_noyau=3, 
                    couche3_nb_neurones=100
                )
R = ReseauClassifGenerique(mon_archi, eta=0.05, alpha=0.1, nb_epoques=40, taille_batch=32)